In [6]:
import numpy as np

In [7]:
def input_timestep(Input, Evaporation_Mean, Precipitation, Temperature):
    
    #Input.Potential_Evaporation::Array{Float64,1} = Evaporation
    Input.Potential_Evaporation_Mean = Evaporation_Mean
    # get the precipitation data of the necessary elevations
    Precipitation_HRU = []
    Temperature_HRU = []
    for i in Input.Elevation_Count:
        Precipitation_HRU.append(Precipitation[i])
        Temperature_HRU.append(Temperature[i])
    Input.Precipitation = Precipitation_HRU
    Input.Temp_Elevation = Temperature_HRU
    
    return Input

In [4]:
def run_model(Area, Evaporation_Mean, Precipitation, Temp,
                bare_input, forest_input, grass_input, rip_input,
                bare_storage, forest_storage, grass_storage, rip_storage, Slowstorage,
                bare_parameters, forest_parameters, grass_parameters, rip_parameters, slow_parameters,
                Total_Elevationbands, Elevation_Percentage):
     # the function takes as input the parameters of each HRU, the inital storage values of each HRU, the inital value of the slow storage
    # KS, ratio riparian, all inputs
    
    #define maximum time
    tmax = len(Precipitation[:,0])
    
    Discharge = np.zeros(tmax)
    WBtotal = np.zeros(tmax)
    Snow_Extend = np.zeros(tmax, Total_Elevationbands)
    
    # store the initial storage values, Does not take into account GW storage!!
    # Assumption_ GW storage is 0 at start
    Initial_Storage_bare = bare_storage.Fast + sum(bare_storage.Interception) + sum(bare_storage.Snow) + bare_storage.Soil
    Initial_Storage_forest = forest_storage.Fast + sum(forest_storage.Interception) + sum(forest_storage.Snow) + forest_storage.Soil
    Initial_Storage_grass = grass_storage.Fast + sum(grass_storage.Interception) + sum(grass_storage.Snow) + grass_storage.Soil
    Initial_Storage_rip = rip_storage.Fast + sum(rip_storage.Interception) + sum(rip_storage.Snow) + rip_storage.Soil
    Initial_Storage = Initial_Storage_bare + Initial_Storage_forest + Initial_Storage_grass + Initial_Storage_rip + Slowstorage
    
    for t in range(0, tmax+1):
        # at each timestep new temp, precipitation and Epot values have to be delivered
        # areas don't change
        # riparian discharge from former timestep has to be used
        # gives the current precipitation, evaporation and temperature
        Evaporation_Mean_Current = Evaporation_Mean[t]
        Precipitation_Current = Precipitation[t, :]
        Temperature_Current = Temp[t, :] 
        
        bare_input = input_timestep(bare_input, Evaporation_Mean_Current, Precipitation_Current, Temperature_Current)
        forest_input = input_timestep(forest_input, Evaporation_Mean_Current, Precipitation_Current, Temperature_Current)
        grass_input = input_timestep(grass_input, Evaporation_Mean_Current, Precipitation_Current, Temperature_Current)
        rip_input = input_timestep(rip_input, Evaporation_Mean_Current, Precipitation_Current, Temperature_Current)
        
        # give new riparian discharge as input for next timestep
        rip_input.Riparian_Discharge = Riparian_Discharge
        # save the fluxes of the current timestep
        Discharge[t] = Total_Discharge/1000 * Area / (3600 * 24) # mm convert it to meter and than * area / seconds in one day
        # OPTIONAL: store the total amount of snow in each elevation (considering whole catchment)
        Snow_Elevations, Snow_Extend[t,:] = snowperelevation(bare_input, forest_input, grass_input, rip_input, bare_storage, forest_storage, grass_storage, rip_storage, Total_Elevationbands)
        #Bare_Snow[t,:]::Array{Float64,1} = bare_storage.Snow
        Snow_Extend[t,:] = Snow_Extend[t,:] / Elevation_Percentage

        WBtotal[t] = WB
                
    # Check Water Balance
    Waterbalance = sum(WBtotal)
    assert Waterbalance <= 10**(-10)
    
    return Discharge, Snow_Extend, Waterbalance


1
2
3
4
